In [1]:
import csv
import cv2
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf
import math

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

training_data = ['./data','./data2', './data3', './data4', './data5']

#generator function for training
def generator(samples, batch_size=1024):
    num_samples = len(samples)
    angle_corr = 0.055 #angle correction value for left and right camera angles
    while 1:
        sklearn.utils.shuffle(samples)
        for offset in range( 0, num_samples, batch_size ):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                image_center = mpimg.imread(batch_sample[0])
                image_left = mpimg.imread(batch_sample[1])
                image_right = mpimg.imread(batch_sample[2])
                #all the camera angles and all of them also flipped (6 images per line)
                angle = float(batch_sample[3])
                images.append(image_center)
                angles.append(angle)
                images.append(cv2.flip(image_center, 1))
                angles.append(-angle)
                images.append(image_left)
                angles.append(angle+angle_corr)
                images.append(cv2.flip(image_left, 1))
                angles.append(-(angle+angle_corr))
                images.append(image_right)
                angles.append(angle-angle_corr)
                images.append(cv2.flip(image_right, 1))
                angles.append(-(angle-angle_corr))
                
            # Return a training batch of size 6*batch_size to model.fit_generator
            X_train = np.array(images)
            y_train = np.array(angles)         
            yield sklearn.utils.shuffle(X_train, y_train)
            
#for the validatation only use the center image (that will be used in the live setup)            
def val_generator(samples, batch_size=1024):
    num_samples = len(samples)
    while 1:
        sklearn.utils.shuffle(samples)
        for offset in range( 0, num_samples, batch_size ):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                image_center = mpimg.imread(batch_sample[0])              
                angle = float(batch_sample[3])
                images.append(image_center)
                angles.append(angle)
                
            # Return a training batch of size 6*batch_size to model.fit_generator
            X_train = np.array(images)
            y_train = np.array(angles)         
            yield sklearn.utils.shuffle(X_train, y_train)            

#reading the training data - filenames and angles
lines=[]
for path in training_data:
    image_path = path+'/IMG/'
    with open(path + '/driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            if line[0] == 'center':
                continue
            steering_angle = float(line[3])
            if steering_angle==0:
                if np.random.randint(4): #dropping randomly 3/4 of the forward angle images (there are too many)
                    continue
            line[0]=image_path + line[0].split('/')[-1]
            line[1]=image_path + line[1].split('/')[-1]
            line[2]=image_path + line[2].split('/')[-1]
            lines.append(line)              
            
train_samples, validation_samples = train_test_split(lines, test_size=0.2)    
print(len(train_samples))
print(len(validation_samples))

batch_size = 64
epochs = 5

# Define generators for training and validation data, to be used with fit_generator below
train_generator = generator(train_samples, batch_size)
validation_generator = val_generator(validation_samples, batch_size)

Using TensorFlow backend.


12174
3044


In [2]:
#The NVIDIA network extended with a cropping layer
model = Sequential()
model.add(Cropping2D(cropping=((55,25), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x: x/255. - 0.5))
model.add(Convolution2D(24, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
model.summary()

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

def calc_samples_per_epoch(array_size, batch_size):
    num_batches = array_size / batch_size
    samples_per_epoch = math.floor(num_batches)
    samples_per_epoch = samples_per_epoch * batch_size
    return samples_per_epoch

epoch_samples=calc_samples_per_epoch(len(train_samples), batch_size)
print(epoch_samples)

history = model.fit_generator(train_generator, 
                              validation_data = validation_generator,
                              samples_per_epoch = len(train_samples)*6, #6 images per line
                              nb_val_samples = len(validation_samples),
                              nb_epoch = epochs, 
                              verbose=1,
                              callbacks=None)
print('Model trained.')
model.save('model.h5')


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_1 (Cropping2D)        (None, 80, 320, 3)    0           cropping2d_input_1[0][0]         
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 80, 320, 3)    0           cropping2d_1[0][0]               
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 38, 158, 24)   1824        lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 17, 77, 36)    21636       convolution2d_1[0][0]            
___________________________________________________________________________________________